# Notes sur la back propagation et chaine rule

Étudiants Rémi Cadène et Mickael Chen

#Introduction

https://github.com/torch/nn/blob/master/Module.lua

Un module torch possède deux fonctions clés :

```lua
function Module:forward(input)
   return self:updateOutput(input)
end

function Module:backward(input, gradOutput, scale)
   scale = scale or 1 -- osef, toujours à 1
   self:updateGradInput(input, gradOutput) -- maj de self.gradInput
   self:accGradParameters(input, gradOutput, scale)
   return self.gradInput
end
```

Vous vous demandez quel est le rapport avec ça ?
$$\frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\theta^m_{i,j}} = \frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\hat{y^m_{j}}} * \frac{\partial\hat{y^m_{j}}}{\partial\theta^m_{i,j}}$$

Et qu'est ce que sont $gradOuput$ et $gradInput$ ? Alors ce tutoriel est fait pour vous.



### Algorithmiquement, pour une full forward-backward pass d'un module linéaire, on a :

In [1]:
require 'nn'

dimInput = 6
dimOutput = 3
learningRate = 1e-2
input = torch.Tensor{1,2,3,4,5,6}
label = torch.Tensor{0,3,2}

modele = nn.Linear(dimInput, dimOutput)
criterion = nn.MSECriterion()

modele:zeroGradParameters()
output = modele:forward(input)
loss = criterion:forward(output, label)
dloss_do = criterion:backward(output, label)
dloss_di = modele:backward(input, dloss_do)
modele:updateParameters(learningRate)

- input est un vecteur de taille dimInput
- output et label sont des vecteurs de tailles dimOutput
- loss est un scalaire
- df_do est un vecteur de taille dimOutput, c'est donc la dérivée du loss en fonction de l'output càd gradOuput
- df_di est un vecteur de taille dimInput, c'est donc la dérivée du loss en fonction de l'input càd gradInput

### De même, pour une full forward-backward pass d'un réseau de neurones à une couche cachée, on a :

In [4]:
dimHidden = 10
module_1 = nn.Linear(dimInput, dimHidden)
module_2 = nn.Tanh()
module_3 = nn.Linear(dimHidden, dimOutput)
criterion = nn.MSECriterion()
module_1:zeroGradParameters()
module_3:zeroGradParameters()
output_m1  = module_1:forward(input)
output_m2  = module_2:forward(output_m1)
output     = module_3:forward(output_m2)
loss       = criterion:forward(output, label)
dloss_do   = criterion:backward(output, label)
dloss_di   = module_3:backward(output_m2, dloss_do)
dloss_dom1 = module_2:backward(output_m1, dloss_di)
dloss_dom2 = module_1:backward(input, dloss_dom1)
module_1:updateParameters(learningRate)
module_3:updateParameters(learningRate)

On remarque notamment que dloss_di est le gradInput du module_3 calculé par ce dernier et qu'il devient le gradOutput du module_2 (précédent). C'est ce processus qu'on appelle "backprop chain-rule".

### Mathématiquement, pour une full forward-backward pass d'un réseau de neurones à une couche cachée, on a :

- soit $x^m_i$ la i-ème entrée du vecteur entrée $x^m$ du module $m$
- soit $x$ le vecteur entrée du premier module $x^1$
- soit $f_\theta(x)$ l'output du modèle
- soit $\bigtriangleup(f_\theta(x), y)$ l'erreur totale du modèle càd le loss
- soit $\theta^m_{i,j}$ le paramètre du module $m$ au rang $i,j$ de la matrice de poids $\theta^m$ 
- soit $\hat{y^m_j}$ la j-ème sortie du vecteur sortie $\hat{y^m}$ du module $m$
- soit $y$ le vecteur label ou sortie du dernier module $\hat{y^3}$
- soit $*$ le produit matriciel

Notons que le terme module fait référence à une couche d'un réseau de neurone et modèle fait référence à toutes les couches. Càd $f_\theta(x)$ fait référence à la sortie du modèle, tandis que $\hat{y^m}$ fait référence à la sortie d'un module. Ainsi la sortie du dernier module est égale à la sortie du modèle. 

Maintenant, on veut calculer pour chaque poids $\theta^m_{i,j}$ du modèle son $gradient$ càd $\frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\theta^m_{i,j}}$.

On rappel, qu'algorithmiquement, ce calcul est effectué lors de l'appel à la fonction $self:accGradParameters(input, gradOutput)$.

##### On sait que $input$ est l'entrée $x^m$, mais qu'est ce que $gradOuput$ à part le $gradInput$ du module suivant ?

Pour comprendre, on décompose : $$\frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\theta^m_{i,j}} = \frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\hat{y^m_{j}}} * \frac{\partial\hat{y^m_{j}}}{\partial\theta^m_{i,j}}$$

On voit apparaître deux termes :
-  $\frac{\partial\hat{y^m_{i,j}}}{\partial\theta^m_{i,j}}$ qui est la dérivée de la sortie du module donc de la fonction $f^m(x^m)$ en fonction de chaque paramètre du module $\theta^m_{i,j}$. Ainsi, $f^m$ peut être la fonction de coût (criterion), ou la multiplication matricielle d'un module linéaire, ou encore la fonction sigmoïde du module $nn.Tanh$ (cette dernière n'ayant pas de paramètres, sa dérivée sera nulle).
- $\frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\hat{y^m_{j}}}$ qui est la dérivée de l'erreur du modèle en fonction de la sortie du module $m$, mais encore de l'entrée du module $m+1$. Ainsi pour obtenir ce terme, nous devons dériver l'erreur du modèle en fonction de l'entrée $x^{m+1}$. Cette dérivée est calculée lors de l'appel à la fonction $self:updateGradInput(input, gradOutput)$ du module suivant.

##### Mais qu'est ce que cette dérivée du loss en fonction de l'entrée du module $m+1$ ?

Pour comprendre, on décompose : $$\frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial{x^{m+1}_i}} = \frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\hat{y^{m+1}_{j}}} * \frac{\partial\hat{y^{m+1}_{j}}}{\partial{x^{m+1}_i}}$$

On retrouve le terme $\frac{\partial\bigtriangleup(f_\theta(x), y)}{\partial\hat{y^{m+1}_{j}}}$ qui est donc le fameux $gradOuput$.



#### Conclusion

Pour résumer, un module $m$ doit faire deux trucs :
1. Calculer son $gradInput$ avec la méthode $updateGradInput(input, gradOutput)$ ($gradOuput$ étant le $gradInput$ du module $m+1$). Ce qui revient à calculer $\frac{\partial\hat{y^{m}_{j}}}{\partial{x^{m}_i}}$, puis à le multiplier par $gradOutput$.
2. Calculer son $gradParameters$ avec la méthode $accGradParameters(input, gradOutput)$. Ce qui revient à calculer $\frac{\partial\hat{y^m_{j}}}{\partial\theta^m_{i,j}}$, puis à le mutliplier par $gradOuput$.

Beware of the reshape.



##### J'espère que cela aura été usefull :) bon courage pour l'implémentation